- https://valencia.opendatasoft.com/api/explore/v2.1/catalog/datasets/estacions-contaminacio-atmosferiques-estaciones-contaminacion-atmosfericas/records?limit=20

In [1]:
from time import sleep
from os import path
import requests
import pandas as pd

In [2]:
def make_request(url, params):
    response = requests.get(url, params=params)
    if response:
        return response.json()
    else:
        raise Exception("error downloading data")

In [3]:
base_url = "https://valencia.opendatasoft.com/api/explore/v2.1/catalog/datasets/estacions-contaminacio-atmosferiques-estaciones-contaminacion-atmosfericas/records"

In [4]:
params = {
    'limit': 0, 
}

n_stations = make_request(base_url, params)['total_count']

print(n_stations) 


11


In [6]:
first_time = True
max_records_per_request = 100

current_time = 0
sleep_time = 60 * 60
total_time = (24 * 60 * 60) + sleep_time

#to test the code
total_time = 1
sleep_time = 1

path_csv = ['res', 'valencia_pollution_dataset.csv']
path_csv_solved = path.join(*path_csv)

In [7]:
while current_time < total_time:
    offset = 0
    pollution_list = []

    while offset < n_stations:
        params = {
            'limit': max_records_per_request,
            'offset': offset,
        }

        sub_list = make_request(base_url, params)['results']
        pollution_list += sub_list

        offset += max_records_per_request

    print(f"Current time = {current_time}, records processed = {len(pollution_list)}")

    df = pd.DataFrame(pollution_list,
                      columns = ['objectid', 'nombre', 'direccion', 'tipozona', 'tipoemisio', 'so2', 'no2', 'o3', 'co', 'pm10', 'pm25', 'fecha_carg', 'calidad_ambiental' ]
                      )

    # display(df)

    df.to_csv(path_csv_solved, sep = ',', header = True if first_time else False, index = False, mode = 'a', na_rep = 'N/A')
    first_time = False
    sleep(sleep_time)

    current_time += sleep_time

Current time = 0, records processed = 11


In [8]:
display(df.head(5))

,objectid,nombre,direccion,tipozona,tipoemisio,so2,no2,o3,co,pm10,pm25,fecha_carg,calidad_ambiental
0,13,Francia,AVDA.FRANCIA,Urbana,Tráfico,1.0,3.0,83.0,0.0,47.0,31.0,2025-06-15T21:00:00+00:00,NaN
1,16,Pista de Silla,PISTA DE SILLA,Urbana,Tráfico,0.0,3.0,91.0,0.0,47.0,26.0,2025-06-15T21:00:00+00:00,NaN
2,17,Universidad Politécnica,POLITÈCNIC,Suburbana,Fondo,5.0,6.0,91.0,NaN,41.0,23.0,2025-06-15T21:00:00+00:00,NaN
3,20,Cabanyal,CABANYAL,Urbana,Fondo,NaN,4.0,NaN,NaN,42.0,22.0,2025-06-15T21:00:00+00:00,NaN
4,21,Olivereta,OLIVERETA,Urbana,Tráfico,NaN,12.0,NaN,NaN,34.0,25.0,2025-06-15T21:00:00+00:00,NaN


In [9]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   objectid           11 non-null     int64  
 1   nombre             11 non-null     object 
 2   direccion          11 non-null     object 
 3   tipozona           11 non-null     object 
 4   tipoemisio         11 non-null     object 
 5   so2                6 non-null      float64
 6   no2                11 non-null     float64
 7   o3                 6 non-null      float64
 8   co                 3 non-null      float64
 9   pm10               9 non-null      float64
 10  pm25               9 non-null      float64
 11  fecha_carg         11 non-null     object 
 12  calidad_ambiental  0 non-null      float64
dtypes: float64(7), int64(1), object(5)
memory usage: 1.2+ KB


None

In [10]:
display(df.describe(include = 'all'))

,objectid,nombre,direccion,tipozona,tipoemisio,so2,no2,o3,co,pm10,pm25,fecha_carg,calidad_ambiental
count,11.000000,11,11,11,11,6.000000,11.000000,6.000000,3.0,9.000000,9.000000,11,0.0
unique,NaN,11,11,2,2,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN
top,NaN,Francia,AVDA.FRANCIA,Urbana,Tráfico,NaN,NaN,NaN,NaN,NaN,NaN,2025-06-15T21:00:00+00:00,NaN
freq,NaN,1,1,9,8,NaN,NaN,NaN,NaN,NaN,NaN,10,NaN
mean,17.000000,NaN,NaN,NaN,NaN,2.666667,5.545455,91.166667,0.0,38.888889,23.666667,NaN,NaN
std,3.316625,NaN,NaN,NaN,NaN,2.250926,3.205110,5.946988,0.0,5.622376,3.807887,NaN,NaN
min,12.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,83.000000,0.0,33.000000,17.000000,NaN,NaN
25%,14.500000,NaN,NaN,NaN,NaN,1.000000,3.500000,88.750000,0.0,34.000000,22.000000,NaN,NaN
50%,17.000000,NaN,NaN,NaN,NaN,2.500000,6.000000,91.000000,0.0,38.000000,23.000000,NaN,NaN
75%,19.500000,NaN,NaN,NaN,NaN,4.750000,6.500000,92.500000,0.0,42.000000,25.000000,NaN,NaN
